In [3]:
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import snowflake.connector
import os

# Thông tin kết nối Snowflake
conn_params = {
    'user': 'dohung24112002',
    'password': 'Doviethung24112002@',
    'account': 'uz36730.ap-southeast-1',
    'warehouse': 'COMPUTE_WH',
    'database': 'AIRBNB',
    'schema': 'DEV'
}

# Tạo kết nối tới Snowflake
conn = snowflake.connector.connect(**conn_params)
print('Connected Successfully!')


Connected Successfully!


In [28]:
# Read recipe inputs
Target_Dataset = dataiku.Dataset("TARGET_DATASET")
Target_Dataset_df = Target_Dataset.get_dataframe()
Target_Dataset_df.head()

customerID gender SeniorCitizen Partner Dependents  tenure PhoneService MultipleLines InternetService       OnlineSecurity         OnlineBackup     DeviceProtection          TechSupport          StreamingTV      StreamingMovies        Contract PaperlessBilling              PaymentMethod  MonthlyCharges  TotalCharges  numAdminTickets  numTechTickets Churn    States        Country
0  7469-LKBCI   Male            No      No         No      16          Yes            No              No  No internet service  No internet service  No internet service  No internet service  No internet service  No internet service        Two year               No    Credit card (automatic)           18.95        326.80                0               0    No  Colorado  United States
1  8091-TTVAX   Male            No     Yes         No      58          Yes           Yes     Fiber optic                   No                   No                  Yes                   No                  Yes                  Yes        One year               No    Credit card (automatic)          100.35       5681.10                0               0    No   Montana  United States
2  0280-XJGEX   Male            No      No         No      49          Yes           Yes     Fiber optic                   No                  Yes                  Yes                   No                  Yes                  Yes  Month-to-month              Yes  Bank transfer (automatic)          103.70       5036.30                5               4   Yes     Maine  United States
3  3655-SNQYZ   Male            No     Yes        Yes      69          Yes           Yes     Fiber optic                  Yes                  Yes                  Yes                  Yes                  Yes                  Yes        Two year               No    Credit card (automatic)          113.25       7895.15                0               0    No   Vermont  United States
4  8191-XWSZG   Male            No      No         No      52          Yes            No              No  No internet service  No internet service  No internet service  No internet service  No internet service  No internet service        One year               No               Mailed check           20.65       1022.95                0               0    No  Delaware  United States

In [29]:
# Bảng đích trong Snowflake
target_table = 'DWH_CHURN_DATASET'

# Đọc dữ liệu hiện có trong Snowflake 
existing_data_query = f"SELECT * FROM {target_table}"
existing_data_df = pd.read_sql(existing_data_query, conn)
print(f"Existing data rows: {len(existing_data_df)}")
existing_data_df.head()

Existing data rows: 0


Empty DataFrame
Columns: [CUSTOMERID, GENDER, SENIORCITIZEN, PARTNER, DEPENDENTS, TENURE, PHONESERVICE, MULTIPLELINES, INTERNETSERVICE, ONLINESECURITY, ONLINEBACKUP, DEVICEPROTECTION, TECHSUPPORT, STREAMINGTV, STREAMINGMOVIES, CONTRACT, PAPERLESSBILLING, PAYMENTMETHOD, MONTHLYCHARGES, TOTALCHARGES, NUMADMINTICKETS, NUMTECHTICKETS, CHURN, STATES, COUNTRY]
Index: []

In [30]:

# Đổi tên cột 'CUSTOMERID' thành 'customerID' trong existing_data_df
existing_data_df = existing_data_df.rename(columns={'CUSTOMERID': 'customerID'})

print(Target_Dataset_df.columns)
print('----------')
print(existing_data_df.columns)

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'numAdminTickets', 'numTechTickets', 'Churn', 'States', 'Country'], dtype='object')
----------
Index(['customerID', 'GENDER', 'SENIORCITIZEN', 'PARTNER', 'DEPENDENTS', 'TENURE', 'PHONESERVICE', 'MULTIPLELINES', 'INTERNETSERVICE', 'ONLINESECURITY', 'ONLINEBACKUP', 'DEVICEPROTECTION', 'TECHSUPPORT', 'STREAMINGTV', 'STREAMINGMOVIES', 'CONTRACT', 'PAPERLESSBILLING', 'PAYMENTMETHOD', 'MONTHLYCHARGES', 'TOTALCHARGES', 'NUMADMINTICKETS', 'NUMTECHTICKETS', 'CHURN', 'STATES', 'COUNTRY'], dtype='object')


In [31]:
# Xác định các bản ghi cần cập nhật hoặc thêm mới
merged_df = pd.merge(Target_Dataset_df, existing_data_df, on='customerID', how='outer', indicator=True)
merged_df.head()

customerID gender SeniorCitizen Partner Dependents  tenure PhoneService MultipleLines InternetService       OnlineSecurity         OnlineBackup     DeviceProtection          TechSupport          StreamingTV      StreamingMovies        Contract PaperlessBilling              PaymentMethod  MonthlyCharges  TotalCharges  numAdminTickets  numTechTickets Churn    States        Country GENDER SENIORCITIZEN PARTNER DEPENDENTS TENURE PHONESERVICE MULTIPLELINES INTERNETSERVICE ONLINESECURITY ONLINEBACKUP DEVICEPROTECTION TECHSUPPORT STREAMINGTV STREAMINGMOVIES CONTRACT PAPERLESSBILLING PAYMENTMETHOD MONTHLYCHARGES TOTALCHARGES NUMADMINTICKETS NUMTECHTICKETS CHURN STATES COUNTRY     _merge
0  7469-LKBCI   Male            No      No         No      16          Yes            No              No  No internet service  No internet service  No internet service  No internet service  No internet service  No internet service        Two year               No    Credit card (automatic)           18.95        326.80                0               0    No  Colorado  United States    NaN           NaN     NaN        NaN    NaN          NaN           NaN             NaN            NaN          NaN              NaN         NaN         NaN             NaN      NaN              NaN           NaN            NaN          NaN             NaN            NaN   NaN    NaN     NaN  left_only
1  8091-TTVAX   Male            No     Yes         No      58          Yes           Yes     Fiber optic                   No                   No                  Yes                   No                  Yes                  Yes        One year               No    Credit card (automatic)          100.35       5681.10                0               0    No   Montana  United States    NaN           NaN     NaN        NaN    NaN          NaN           NaN             NaN            NaN          NaN              NaN         NaN         NaN             NaN      NaN              NaN           NaN            NaN          NaN             NaN            NaN   NaN    NaN     NaN  left_only
2  0280-XJGEX   Male            No      No         No      49          Yes           Yes     Fiber optic                   No                  Yes                  Yes                   No                  Yes                  Yes  Month-to-month              Yes  Bank transfer (automatic)          103.70       5036.30                5               4   Yes     Maine  United States    NaN           NaN     NaN        NaN    NaN          NaN           NaN             NaN            NaN          NaN              NaN         NaN         NaN             NaN      NaN              NaN           NaN            NaN          NaN             NaN            NaN   NaN    NaN     NaN  left_only
3  3655-SNQYZ   Male            No     Yes        Yes      69          Yes           Yes     Fiber optic                  Yes                  Yes                  Yes                  Yes                  Yes                  Yes        Two year               No    Credit card (automatic)          113.25       7895.15                0               0    No   Vermont  United States    NaN           NaN     NaN        NaN    NaN          NaN           NaN             NaN            NaN          NaN              NaN         NaN         NaN             NaN      NaN              NaN           NaN            NaN          NaN             NaN            NaN   NaN    NaN     NaN  left_only
4  8191-XWSZG   Male            No      No         No      52          Yes            No              No  No internet service  No internet service  No internet service  No internet service  No internet service  No internet service        One year               No               Mailed check           20.65       1022.95                0               0    No  Delaware  United States    NaN           NaN     NaN        NaN    NaN          NaN           NaN             NaN            NaN          NaN              NaN         NaN         NaN     

In [32]:
to_update_df = merged_df[merged_df['_merge'] == 'both']
to_insert_df = merged_df[merged_df['_merge'] == 'left_only']
to_insert_df.head()

customerID gender SeniorCitizen Partner Dependents  tenure PhoneService MultipleLines InternetService       OnlineSecurity         OnlineBackup     DeviceProtection          TechSupport          StreamingTV      StreamingMovies        Contract PaperlessBilling              PaymentMethod  MonthlyCharges  TotalCharges  numAdminTickets  numTechTickets Churn    States        Country GENDER SENIORCITIZEN PARTNER DEPENDENTS TENURE PHONESERVICE MULTIPLELINES INTERNETSERVICE ONLINESECURITY ONLINEBACKUP DEVICEPROTECTION TECHSUPPORT STREAMINGTV STREAMINGMOVIES CONTRACT PAPERLESSBILLING PAYMENTMETHOD MONTHLYCHARGES TOTALCHARGES NUMADMINTICKETS NUMTECHTICKETS CHURN STATES COUNTRY     _merge
0  7469-LKBCI   Male            No      No         No      16          Yes            No              No  No internet service  No internet service  No internet service  No internet service  No internet service  No internet service        Two year               No    Credit card (automatic)           18.95        326.80                0               0    No  Colorado  United States    NaN           NaN     NaN        NaN    NaN          NaN           NaN             NaN            NaN          NaN              NaN         NaN         NaN             NaN      NaN              NaN           NaN            NaN          NaN             NaN            NaN   NaN    NaN     NaN  left_only
1  8091-TTVAX   Male            No     Yes         No      58          Yes           Yes     Fiber optic                   No                   No                  Yes                   No                  Yes                  Yes        One year               No    Credit card (automatic)          100.35       5681.10                0               0    No   Montana  United States    NaN           NaN     NaN        NaN    NaN          NaN           NaN             NaN            NaN          NaN              NaN         NaN         NaN             NaN      NaN              NaN           NaN            NaN          NaN             NaN            NaN   NaN    NaN     NaN  left_only
2  0280-XJGEX   Male            No      No         No      49          Yes           Yes     Fiber optic                   No                  Yes                  Yes                   No                  Yes                  Yes  Month-to-month              Yes  Bank transfer (automatic)          103.70       5036.30                5               4   Yes     Maine  United States    NaN           NaN     NaN        NaN    NaN          NaN           NaN             NaN            NaN          NaN              NaN         NaN         NaN             NaN      NaN              NaN           NaN            NaN          NaN             NaN            NaN   NaN    NaN     NaN  left_only
3  3655-SNQYZ   Male            No     Yes        Yes      69          Yes           Yes     Fiber optic                  Yes                  Yes                  Yes                  Yes                  Yes                  Yes        Two year               No    Credit card (automatic)          113.25       7895.15                0               0    No   Vermont  United States    NaN           NaN     NaN        NaN    NaN          NaN           NaN             NaN            NaN          NaN              NaN         NaN         NaN             NaN      NaN              NaN           NaN            NaN          NaN             NaN            NaN   NaN    NaN     NaN  left_only
4  8191-XWSZG   Male            No      No         No      52          Yes            No              No  No internet service  No internet service  No internet service  No internet service  No internet service  No internet service        One year               No               Mailed check           20.65       1022.95                0               0    No  Delaware  United States    NaN           NaN     NaN        NaN    NaN          NaN           NaN             NaN            NaN          NaN              NaN         NaN         NaN     

In [33]:
columns_to_keep = Target_Dataset_df.columns
to_insert_df = merged_df[columns_to_keep]
to_insert_df.head()

customerID gender SeniorCitizen Partner Dependents  tenure PhoneService MultipleLines InternetService       OnlineSecurity         OnlineBackup     DeviceProtection          TechSupport          StreamingTV      StreamingMovies        Contract PaperlessBilling              PaymentMethod  MonthlyCharges  TotalCharges  numAdminTickets  numTechTickets Churn    States        Country
0  7469-LKBCI   Male            No      No         No      16          Yes            No              No  No internet service  No internet service  No internet service  No internet service  No internet service  No internet service        Two year               No    Credit card (automatic)           18.95        326.80                0               0    No  Colorado  United States
1  8091-TTVAX   Male            No     Yes         No      58          Yes           Yes     Fiber optic                   No                   No                  Yes                   No                  Yes                  Yes        One year               No    Credit card (automatic)          100.35       5681.10                0               0    No   Montana  United States
2  0280-XJGEX   Male            No      No         No      49          Yes           Yes     Fiber optic                   No                  Yes                  Yes                   No                  Yes                  Yes  Month-to-month              Yes  Bank transfer (automatic)          103.70       5036.30                5               4   Yes     Maine  United States
3  3655-SNQYZ   Male            No     Yes        Yes      69          Yes           Yes     Fiber optic                  Yes                  Yes                  Yes                  Yes                  Yes                  Yes        Two year               No    Credit card (automatic)          113.25       7895.15                0               0    No   Vermont  United States
4  8191-XWSZG   Male            No      No         No      52          Yes            No              No  No internet service  No internet service  No internet service  No internet service  No internet service  No internet service        One year               No               Mailed check           20.65       1022.95                0               0    No  Delaware  United States

In [34]:
### UPDATE STEP ###
columns_to_keep = existing_data_df.columns
to_update_df = to_update_df[columns_to_keep]

print("DataFrame to_update_df after column selection:")
print(to_update_df.head())

DataFrame to_update_df after column selection:
Empty DataFrame
Columns: [customerID, GENDER, SENIORCITIZEN, PARTNER, DEPENDENTS, TENURE, PHONESERVICE, MULTIPLELINES, INTERNETSERVICE, ONLINESECURITY, ONLINEBACKUP, DEVICEPROTECTION, TECHSUPPORT, STREAMINGTV, STREAMINGMOVIES, CONTRACT, PAPERLESSBILLING, PAYMENTMETHOD, MONTHLYCHARGES, TOTALCHARGES, NUMADMINTICKETS, NUMTECHTICKETS, CHURN, STATES, COUNTRY]
Index: []


In [35]:
print(f"Records to update: {len(to_update_df)}, Records to insert: {len(to_insert_df)}")

Records to update: 0, Records to insert: 3987


In [36]:
print(to_update_df.empty)

True


In [25]:
# Thay thế các giá trị NaN trong to_update_df và to_insert_df
to_update_df = to_update_df.fillna('')
to_insert_df = to_insert_df.fillna('')

# In nội dung của DataFrame để xác minh
print("DataFrame to_update_df:")
print(to_update_df.head())
print("DataFrame to_insert_df:")
print(to_insert_df.head())

In [27]:
to_insert_df = to_insert_df.fillna('')

In [0]:


### SUMMARY ###
print(f"Records to update: {len(to_update_df)}, Records to insert: {len(to_insert_df)}")

### UPDATE STEP ###
columns_to_keep = existing_data_df.columns
to_update_df = to_update_df[columns_to_keep]

print("DataFrame to_update_df after column selection:")
print(to_update_df.head())

### INSERT STEP ###
columns_to_keep = Target_Dataset_df.columns
to_insert_df = to_insert_df[columns_to_keep]

print("DataFrame to_insert_df after column selection:")
print(to_insert_df.head())